Formula 1

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt


# 1 Gather Data
We want to...

In [49]:
data_dir_path = "../data/"
# Read the csv files
results = pd.read_csv(data_dir_path + "results.csv", na_values=np.nan)
driver_standings = pd.read_csv(data_dir_path + "driver_standings.csv", na_values=np.nan)
constructor_standings = pd.read_csv(
    data_dir_path + "constructor_standings.csv", na_values=np.nan
)
raceYears = pd.read_csv(data_dir_path + "races.csv", na_values=np.nan)
names = pd.read_csv(data_dir_path + "drivers.csv", na_values=np.nan)

# Filter columns
raceYears = raceYears[["raceId", "year"]]
raceYears = raceYears[raceYears["year"] >= 2010]  # Only data after 2010
results = results[["raceId", "driverId", "constructorId", "grid", "position"]]
driver_standings = driver_standings[["raceId", "driverId", "position"]]
constructor_standings = constructor_standings[["raceId", "constructorId", "position"]]
names = names[["driverId", "driverRef"]]

# rename because other csv also have position
driver_standings = driver_standings.rename(columns={"position": "driverStanding"})
constructor_standings = constructor_standings.rename(
    columns={"position": "constructorStanding"}
)

year_driver_standing = pd.merge(raceYears, driver_standings, on=["raceId"], how="inner")

results_driver_standings = pd.merge(
    results, year_driver_standing, on=["raceId", "driverId"], how="inner"
)

joined_data = pd.merge(
    results_driver_standings,
    constructor_standings,
    on=["raceId", "constructorId"],
    how="inner",
)

joined_data = pd.merge(joined_data, names, on=["driverId"], how="inner")
joined_data.drop(columns=["raceId", "constructorId", "driverId"], inplace=True)
joined_data.replace(to_replace="\\N", value=20, inplace=True)
joined_data["position"] = joined_data["position"].astype(int)


# Divide by train data and test data
train_data = joined_data[joined_data["year"] <= 2019]
test_data = joined_data[joined_data["year"] > 2019]

# Drop year before calculating the mean for every driver
train_data.drop(columns=["year"], inplace=True)
test_data.drop(columns=["year"], inplace=True)

# Calculate mean values for every driver
# train_data = train_data.groupby(["driverRef"]).mean()
# test_data = test_data.groupby(["driverRef"]).mean()

X_train = train_data[["grid", "driverStanding", "constructorStanding"]]
Y_train = train_data[["position"]]


/var/folders/5x/mccc1ybs12ng6r2m2lcksrtm0000gn/T/ipykernel_1515/3268364427.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_data.drop(columns=["year"], inplace=True)
/var/folders/5x/mccc1ybs12ng6r2m2lcksrtm0000gn/T/ipykernel_1515/3268364427.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data.drop(columns=["year"], inplace=True)


2 Standardise and Scale

In [ ]:
scaler = StandardScaler()
